In [1]:
import os
import pyspark
import numpy as np
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType

In [2]:
def get_data(query):
    """
    Returns the pySpark DataFrame
    
    Parameters:
        query (str): The input HQL query
        
    Returns:
        DataFrame (pySpark)
    """
    try:
        return sessn.sql(query)
    except Exception as e:
        print(e)

In [3]:
def get_eda(data, feature_list, feature_family=None):
    """
    Perform EDA on the provided dataset basis on its family i.e categorical or continuos

    Parameters:
        data: DataFrame (pySpark)
        feature_list (list)
        feature_family: continuous or categorical, default:None

    Returns:
        DataFrame (pySpark)
    """
    if feature_family == "continuos":
        from pyspark.sql.functions import isnan, when, count, col
        null_count_df = data.select(
            [count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in feature_list]).toPandas()
        null_count_df.index = ['MissingValue']
        describe_df = data.select(feature_list).describe().toPandas()
        describe_df = describe_df.set_index('summary')
        describe_df = describe_df.append(null_count_df) 
        describe_df = describe_df.reset_index()
        return describe_df
    if feature_family == "categorical":
        from pyspark.sql.functions import trim
        data_cat = data.select(feature_list)
        cat_null_count = []
        freq_dist = {}
        for col_ in data_cat.columns:
            print("Processing {}".format(col_))
            data_cat = data_cat.withColumn(col_, trim(data_cat[col_]))
            freq_dist_df = data_cat.groupby(col_).count().toPandas()
            count = data_cat.filter((data_cat[col_] == "")|(data_cat[col_].isNull())).count()
            cat_null_count.append((col_, count))
            freq_dist[col_] = freq_dist_df
        cat_null_count_df = pd.DataFrame(cat_null_count, columns=['Feature', 'MissingValue'])
        total_lines = data_cat.count()
        cat_null_count_df['%MissingValue'] = (cat_null_count_df['MissingValue'] / total_lines) * 100
        cat_null_count_df['%MissingValue'] = cat_null_count_df['%MissingValue'].round(2)
        return freq_dist, cat_null_count_df
    else:
        print("Only two values are possible: \n1. continuos \n2. categorical")

In [20]:
def control_limit(data, c, k):
    print("Working on {}".format(c))
    col_list = ['servicing_npi']
    col_list.append(c)
    temp_df = data.select(col_list)
    temp_df.createOrReplaceTempView("t1")
    c_q1 = c + "_Q1"
    c_q3 = c + "_Q3"
    c_iqr = c + "_IQR"
    c_ucl = c + "_UCL"
    c_lcl = c + "_LCL"
    query =  "select servicing_npi, percentile({}, 0.25) as {}, percentile({}, 0.75) as {} from t1 group by servicing_npi".format(c, c_q1, c, c_q3)
    query = " ".join(query.split())
    op = get_data(query)
    op = op.withColumn(c_iqr, op[c_q3] - op[c_q1])
    op = op.withColumn(c_lcl, op[c_q1] - (op[c_iqr]*k))
    op = op.withColumn(c_ucl, op[c_q3] + (op[c_iqr]*k))
    op = op.select(['servicing_npi', c_ucl, c_lcl])
    return op

In [5]:
# Configuration
home_path = "/mapr/datalake/uhc/ei/pi_ara/sherlock/development/HPP-Provider-Signature-Monitoring/"
# Input
feature_engineered_table = "pi_usrs.hpp_ps_signal_data"
# Output
outlier_removed_table = "pi_usrs.hpp_ps_modelling_data"

In [6]:
try:
    os.mkdir(home_path)
except:
    print("Directory Exist")
os.chdir(home_path)

Directory Exist


In [7]:
try:
    sc.stop()
except Exception as e:
    print(e)
try:
    sessn.stop()
except Exception as e:
    print(e)
try:
    sc.SparkContext.stop()
except Exception as e:
    print(e)

name 'sessn' is not defined
'SparkContext' object has no attribute 'SparkContext'


In [8]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '12g'), 
                                   ('spark.executor.instances', '20'),
                                   ('spark.executor.cores', '4'), 
                                   ('spark.driver.cores','4'), 
                                   ('spark.driver.memory','16g'),
                                   ('spark.yarn.queue', 'araadh_q1.arapi_sq1'),
                                   ('spark.yarn.executor.memoryOverhead', '2g')])
sessn = SparkSession.builder.appName("HPP").config(conf=conf).enableHiveSupport().getOrCreate().newSession()
sessn

##### Data Acquisition

In [21]:
base_data = sessn.table(feature_engineered_table)

In [22]:
total_clm_cnt = base_data.count()

In [23]:
group_level = ["servicing_npi", "srk_clm_id_rj"]

In [24]:
col_to_work = [c for c in base_data.columns if "ind" not in c]
signal_list = list(set(col_to_work)-set(group_level))

In [27]:
data_to_work = base_data.select(col_to_work)
for c in signal_list:
    c_ucl = c + "_UCL"
    c_lcl = c + "_LCL"
    c_flg = c + "_flag"
    op = control_limit(data_to_work, c, 2)
    data_to_work = data_to_work.join(op, on = "servicing_npi", how = "inner")
    data_to_work = data_to_work.withColumn(c_flg, F.when(((data_to_work[c] <= data_to_work[c_ucl]) & (data_to_work[c] >= data_to_work[c_lcl])), F.lit(1)).otherwise(F.lit(0)))
    data_to_work = data_to_work.drop(c_ucl)
    data_to_work = data_to_work.drop(c_lcl)

Working on proc_code_unusual
Working on billed_amt_sum
Working on clm_unit_billed
Working on clm_ln_cnt
Working on pos_unsual


In [28]:
flag_col = [c for c in data_to_work.columns if "_flag" in c] 
feature_col = [c.replace("_flag","") for c in flag_col]

In [29]:
data_to_work_outlier_removed = data_to_work.filter((F.col('billed_amt_sum_flag') == F.lit(1)) &
                                                    (F.col('clm_ln_cnt_flag') == F.lit(1)) &
                                                    (F.col('clm_unit_billed_flag') == F.lit(1)) &
                                                    (F.col('proc_code_unusual_flag') == F.lit(1)) &
                                                    (F.col('pos_unsual_flag') == F.lit(1)))

In [30]:
data_to_work_outlier_removed = data_to_work_outlier_removed.select(col_to_work)

In [31]:
col_for_ind = list(set(base_data.columns)-set(signal_list))
base_data_ind = base_data.select(col_for_ind)

In [32]:
modelling_data = data_to_work_outlier_removed.join(base_data_ind, on = group_level, how = "inner")

In [37]:
modelling_data.write.mode("overwrite").saveAsTable(outlier_removed_table)

##### Sanity Check

In [38]:
query = "select * from " + outlier_removed_table
output_df = get_data(query)
pd.DataFrame(output_df.dtypes)

0       1
0       servicing_npi  string
1       srk_clm_id_rj  string
2      billed_amt_sum  double
3          clm_ln_cnt  bigint
4     clm_unit_billed  double
5   proc_code_unusual  double
6          pos_unsual  double
7         em_proc_ind     int
8           mod91_ind     int
9           modTC_ind     int
10          mod59_ind     int
11  high_cst_proc_ind     int
12         emgncy_ind     int

In [39]:
output_df_lc = output_df.count()
print("{} claim present in processed data".format(output_df_lc))

1004750 claim present in processed data


In [42]:
drop_clm = total_clm_cnt-output_df_lc
drop_clm_percntg = (drop_clm*100)/total_clm_cnt  

In [40]:
continuos_var = [c[0] for c in output_df.dtypes if (c[1]=='double') | (c[1]=='int') | (c[1]=='bigint')]

In [41]:
eda_continuos = get_eda(output_df, continuos_var, "continuos")
eda_continuos

index      billed_amt_sum          clm_ln_cnt     clm_unit_billed  \
0         count             1004750             1004750             1004750   
1          mean  186.38817566558726   2.329451107240607  2.4029544264742473   
2        stddev  157.79870653295694  2.3897250657959783   2.868500602647453   
3           min                 0.0                   1                 0.0   
4           max              9764.0                  37               351.0   
5  MissingValue                   0                   0                   0   

     proc_code_unusual           pos_unsual          em_proc_ind  \
0              1004750              1004750              1004750   
1  0.12816536451853633   0.9308589350584673  0.37564667827817866   
2  0.15338503587316743  0.18639471418508183  0.48428967034663495   
3                  0.0                  0.0                    0   
4                  1.0                  1.0                    1   
5                    0                    0                    0   

              mod91_ind              modTC_ind             mod59_ind  \
0               1004750                1004750               1004750   
1  4.986315003732271E-4  0.0011803931326200548  0.039580990296093556   
2   0.02232450140601792    0.03433658366960403   0.19497275024457336   
3                     0                      0                     0   
4                     1                      1                     1   
5                     0                      0                     0   

      high_cst_proc_ind             emgncy_ind  
0               1004750                1004750  
1  9.952724558347848E-7  1.1545160487683504E-4  
2  9.976334275848947E-4   0.010744225923556195  
3                     0                      0  
4                     1                      1  
5                     0                      0